
# MMF1921/MIE377 - Backtesting Template

The purpose of this program is to provide a template with which to develop Project 2. The project requires you to test different models  (and/or different model combinations) to create an asset management algorithm.

This template will be used by the instructor and TA to assess your trading algorithm using different datasets.

# PLEASE DO NOT MODIFY THIS TEMPLATE (for Project submission purposes)

# 1. Read input files

In [1]:
import time
import math
from scipy.stats import gmean
import matplotlib.pyplot as plt
from services.project_function import *
import pandas as pd

adjClose = pd.read_csv("MIE377_AssetPrices_3.csv", index_col=0)
factorRet = pd.read_csv("MIE377_FactorReturns_3.csv", index_col=0)

In [ ]:
adjClose.index = pd.to_datetime(adjClose.index)
factorRet.index = pd.to_datetime(factorRet.index)

In [ ]:
# Initial budget to invest ($100,000)
initialVal = 100000

# Length of investment period (in months)
investPeriod = 6

# divide the factor returns by  100
factorRet = factorRet/100

#rf and factor returns
riskFree = factorRet['RF']
factorRet = factorRet.loc[:,factorRet.columns != 'RF'];


In [ ]:
#Identify the tickers and the dates
tickers = adjClose.columns
dates   = factorRet.index

In [ ]:
# Calculate the stocks monthly excess returns
# pct change and drop the first null observation
returns = adjClose.pct_change(1).iloc[1:, :]
returns = returns  - np.diag(riskFree.values) @ np.ones_like(returns.values)
# Align the price table to the asset and factor returns tables by discarding the first observation.
adjClose = adjClose.iloc[1:,:]

In [ ]:
assert adjClose.index[0] == returns.index[0]
assert adjClose.index[0] == factorRet.index[0]

# 2. Run your program

This section will run your Project1_Function in a loop. The data will be loaded progressively as a growing window of historical observations.
Rebalancing will take place after every loop

In [ ]:
# Start of out-of-sample test period
testStart = returns.index[0] + pd.offsets.DateOffset(years=5)

#End of the first investment period
testEnd = testStart + pd.offsets.DateOffset(months=investPeriod) -  pd.offsets.DateOffset(days = 1)

# End of calibration period
calEnd = testStart -  pd.offsets.DateOffset(days = 1)

# Total number of investment periods
NoPeriods = math.ceil((returns.index[-1].to_period('M') - testStart.to_period('M')).n / investPeriod)

# Number of assets
n  = len(tickers)

# Preallocate space for the portfolio weights (x0 will be used to calculate
# the turnover rate)
x  = np.zeros([n, NoPeriods])
x0 = np.zeros([n, NoPeriods])

# Preallocate space for the portfolio per period value and turnover
currentVal = np.zeros([NoPeriods, 1])
turnover   = np.zeros([NoPeriods, 1])

#Initiate counter for the number of observations per investment period
toDay = 0

# Measure runtime: start the clock
start_time = time.time()

# Empty list to measure the value of the portfolio over the period
portfValue = []

for t in range(NoPeriods):
    # Subset the returns and factor returns corresponding to the current calibration period.
    periodReturns = returns[returns.index <= calEnd]
    periodFactRet = factorRet[factorRet.index <= calEnd]

    current_price_idx = (calEnd - pd.offsets.DateOffset(months=1) <= adjClose.index)&(adjClose.index <= calEnd)
    currentPrices = adjClose[current_price_idx]

    # Subset the prices corresponding to the current out-of-sample test period.
    periodPrices_idx = (testStart <= adjClose.index)&(adjClose.index <= testEnd)
    periodPrices = adjClose[periodPrices_idx]

    assert len(periodPrices) == investPeriod
    assert len(currentPrices) == 1
    # Set the initial value of the portfolio or update the portfolio value
    if t == 0:
        currentVal[0] = initialVal
    else:
        currentVal[t] = currentPrices @  NoShares.values.T
        #Store the current asset weights (before optimization takes place)
        x0[:,t] = currentPrices.values*NoShares.values/currentVal[t]

    #----------------------------------------------------------------------
    # Portfolio optimization
    # You must write code your own algorithmic trading function
    # The project function is in the services folder
    # Take in the period returns and period factor returns and produce
    # an allocation
    #----------------------------------------------------------------------
    x[:,t] = project_function(periodReturns, periodFactRet)

    #Calculate the turnover rate
    if t > 0:
        turnover[t] = np.sum( np.abs( x[:,t] - x0[:,t] ) )

    # Number of shares your portfolio holds per stock
    NoShares = x[:,t]*currentVal[t]/currentPrices

    # Update counter for the number of observations per investment period
    fromDay = toDay
    toDay   = toDay + len(periodPrices)

    # Weekly portfolio value during the out-of-sample window
    portfValue.append(periodPrices@ NoShares.values.T)

    # Update your calibration and out-of-sample test periods
    testStart = testStart + pd.offsets.DateOffset(months=investPeriod)
    testEnd   = testStart + pd.offsets.DateOffset(months=investPeriod) - pd.offsets.DateOffset(days=1)
    calEnd    = testStart - pd.offsets.DateOffset(days=1)

portfValue = pd.concat(portfValue, axis = 0)
end_time = time.time()

# 3. Results

In [ ]:
#--------------------------------------------------------------------------
# 3.1 Calculate the portfolio average return, standard deviation, Sharpe ratio and average turnover.
#-----------------------------------------------------------------------
# Calculate the observed portfolio returns
portfRets = portfValue.pct_change(1).iloc[1:,:]

# Calculate the portfolio excess returns
portfExRets = portfRets.subtract(riskFree[(riskFree.index >= portfRets.index[0])&(riskFree.index <= portfRets.index[-1])], axis = 0)

# Calculate the portfolio Sharpe ratio
SR = ((portfExRets + 1).apply(gmean, axis=0) - 1)/portfExRets.std()

# Calculate the average turnover rate
avgTurnover = np.mean(turnover[1:])

#Print Sharpe ratio and Avg. turnover to the console
print("Elasped time is "+ str(end_time - start_time) + ' seconds')
print('Sharpe ratio: ', str(SR[0]))
print('Avg. turnover: ', str(avgTurnover))

In [ ]:
#--------------------------------------------------------------------------
# 3.2 Portfolio wealth evolution plot
#--------------------------------------------------------------------------
# Calculate the dates of the out-of-sample period

fig = plt.figure(1)
portfValue.plot(title = 'Portfolio wealth evolution',
                ylabel = 'Total wealth',
                figsize = (6, 3),
                legend = False)
plt.savefig("images/wealth.svg")

#--------------------------------------------------------------------------
# 3.3 Portfolio weights plot
#--------------------------------------------------------------------------
# Portfolio weights

fig2 = plt.figure(2);
x[x < 0] = 0
weights = pd.DataFrame(x, index = tickers)

weights.columns = [col + 1 for col in weights.columns]
weights.T.plot.area(title = 'Portfolio weights',
                  ylabel = 'Weights', xlabel = 'Rebalance Period',
                  figsize = (6, 3),
                  legend = True, stacked = True)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.savefig("images/weights.svg");
#
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# % Program End

In [ ]:
def run_strategy(num_periods,lambda_RP, lamda_S, z, model):
    testStart = returns.index[0] + pd.offsets.DateOffset(years=5)
    testEnd = testStart + pd.offsets.DateOffset(months=investPeriod) - pd.offsets.DateOffset(days=1)
    calEnd = testStart - pd.offsets.DateOffset(days=1)

    NoPeriods = math.ceil((returns.index[-1].to_period('M') - testStart.to_period('M')).n / investPeriod)
    n = len(tickers)

    x = np.zeros([n, NoPeriods])
    x0 = np.zeros([n, NoPeriods])
    currentVal = np.zeros([NoPeriods, 1])
    turnover = np.zeros([NoPeriods, 1])
    toDay = 0
    portfValue = []

    for t in range(NoPeriods):
        periodReturns = returns[returns.index <= calEnd]
        periodFactRet = factorRet[factorRet.index <= calEnd]

        current_price_idx = (calEnd - pd.offsets.DateOffset(months=1) <= adjClose.index)&(adjClose.index <= calEnd)
        currentPrices = adjClose[current_price_idx]

        periodPrices_idx = (testStart <= adjClose.index)&(adjClose.index <= testEnd)
        periodPrices = adjClose[periodPrices_idx]

        if len(periodPrices) != investPeriod or len(currentPrices) != 1:
            break  # Skip this period if data is missing

        if t == 0:
            currentVal[0] = initialVal
        else:
            currentVal[t] = currentPrices @ NoShares.values.T
            x0[:,t] = currentPrices.values * NoShares.values / currentVal[t]

        x[:,t] = grid_search_function(periodReturns, periodFactRet, num_periods,lambda_RP, lamda_S, z, model)

        if t > 0:
            turnover[t] = np.sum(np.abs(x[:,t] - x0[:,t]))

        NoShares = x[:,t] * currentVal[t] / currentPrices

        fromDay = toDay
        toDay = toDay + len(periodPrices)

        portfValue.append(periodPrices @ NoShares.values.T)

        testStart = testStart + pd.offsets.DateOffset(months=investPeriod)
        testEnd = testStart + pd.offsets.DateOffset(months=investPeriod) - pd.offsets.DateOffset(days=1)
        calEnd = testStart - pd.offsets.DateOffset(days=1)

    if len(portfValue) == 0:
        return -np.inf, np.nan  # Invalid run

    portfValue = pd.concat(portfValue, axis=0)
    portfRets = portfValue.pct_change(1).iloc[1:,:]
    portfExRets = portfRets.subtract(riskFree[(riskFree.index >= portfRets.index[0]) & (riskFree.index <= portfRets.index[-1])], axis=0)

    SR = ((portfExRets + 1).apply(gmean, axis=0) - 1) / portfExRets.std()
    avgTurnover = np.mean(turnover[1:])

    return SR[0], avgTurnover


In [1]:
import pandas as pd
import numpy as np
import math
from scipy.stats import gmean

best_results = {}

# Iterate over models (0, 1, 2)
for model in range(3):
    best_SR = -np.inf
    best_params = {}

    # Model 0 (Risk Parity): Loop through lambda values
    if model == 0:
        for lambda_RP in [0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2]:
            for num_periods in [30, 36, 40]:
                print(f"\nTesting model {model}, lambda_RP={lambda_RP}, num_periods={num_periods}")
                SR, turnover = run_strategy(num_periods=num_periods, lambda_RP=lambda_RP, lamda_S=None, z=None, model=model)
                print(f"--> SR = {SR:.4f}, Turnover = {turnover:.4f}")
                if SR > best_SR:
                    best_SR = SR
                    best_params = {
                        'model': model,
                        'lambda_RP': lambda_RP,
                        'num_periods': num_periods,
                        'SR': SR,
                        'turnover': turnover
                    }

    # Model 1 (Sharpe): Loop through lambda values
    elif model == 1:
        for lamda_S in [0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2]:
            for num_periods in [30, 36, 40]:
                print(f"\nTesting model {model}, lambda_S={lamda_S}, num_periods={num_periods}")
                SR, turnover = run_strategy(num_periods=num_periods, lambda_RP=None, lamda_S=lamda_S, z=None, model=model)
                print(f"--> SR = {SR:.4f}, Turnover = {turnover:.4f}")
                if SR > best_SR:
                    best_SR = SR
                    best_params = {
                        'model': model,
                        'lambda_S': lamda_S,
                        'num_periods': num_periods,
                        'SR': SR,
                        'turnover': turnover
                    }

    # Model 2 (Mix Method): Loop through lambda values and z
    elif model == 2:
        for lambda_RP in [0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2]:
            for lamda_S in [10, 12.5, 15, 17.5, 20]:
                for z in [0.25, 0.5, 0.75]:
                    for num_periods in [30, 36, 40]:
                        print(f"\nTesting model {model}, lambda_RP={lambda_RP}, lambda_S={lamda_S}, z={z}, num_periods={num_periods}")
                        SR, turnover = run_strategy(num_periods=num_periods, lambda_RP=lambda_RP, lamda_S=lamda_S, z=z, model=model)
                        print(f"--> SR = {SR:.4f}, Turnover = {turnover:.4f}")
                        if SR > best_SR:
                            best_SR = SR
                            best_params = {
                                'model': model,
                                'lambda_RP': lambda_RP,
                                'lambda_S': lamda_S,
                                'z': z,
                                'num_periods': num_periods,
                                'SR': SR,
                                'turnover': turnover
                            }

    best_results[model] = best_params


# Prepare data for a nice table
table_data = []
for model, result in best_results.items():
    table_data.append({
        'Model': model,
        'lambda_RP': result.get('lambda_RP', 'N/A'),
        'lambda_S': result.get('lambda_S', 'N/A'),
        'z': result.get('z', 'N/A'),
        'num_periods': result['num_periods'],
        'SR': result['SR'],
        'Turnover': result['turnover']
    })

# Convert to DataFrame for better readability
df_results = pd.DataFrame(table_data)

# Display the results in a table format
print("\nBest Results Summary:")
print(df_results)



Testing model 0, lambda_RP=0, num_periods=30


NameError: name 'run_strategy' is not defined